In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    # ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD" and attack_name!= "DF":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 1% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_1, _ , y_1 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.01, stratify=y_label_0_1, random_state=42)


In [6]:
second_encoder = LabelEncoder()
second_encoder.fit(y_1)

y_1 = second_encoder.transform(y_1)
for index, class_label in enumerate(second_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '0' encode 0
defense '1' encode 1
defense '2' encode 2
defense '3' encode 3
defense '4' encode 4
defense '5' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9


In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt1 = DecisionTreeClassifier()
dt1.fit(X_1, y_1)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb1.predict(X)
    y_pred = second_encoder.inverse_transform(y_pred)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude3Attack/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude3Attack/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [    87    114   1250   1510      6    138     26 110810     19     11]
(87, 56) (87,)
Save 0 to baseline_Def1.npy
(114, 56) (114,)
Save 1 to baseline_Def2.npy
(1250, 56) (1250,)
Save 2 to baseline_Def3.npy
(1510, 56) (1510,)
Save 3 to baseline_Def4.npy
(6, 56) (6,)
Save 4 to baseline_Def5.npy
(138, 56) (138,)
Save 5 to baseline_Def6.npy
(26, 56) (26,)
Save 7 to baseline_Def8.npy
(110810, 56) (110810,)
Save 8 to baseline_Def9.npy
(19, 56) (19,)
Save 9 to baseline_Def10.npy
(11, 56) (11,)
Save 10 to baseline_Def11.npy
Execution Time: 0.144233 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [225056    170    365   2673    562   5374     85 107601     11     16]
(225056, 56) (225056,)
Save 0 to BIM_Def1.npy
(170, 56) (170,)
Save 1 to BIM_Def2.npy
(365, 56) (365,)
Save 2 to BIM_Def3.npy
(2673, 56) (2673,)
Save 3 to BIM_Def4.npy
(562, 56) (562,)
Save 4 to BIM_Def5.npy
(5374, 56) (5374,)
Save 5 to BIM_Def6.npy
(85, 56) (85,)
Save 7 to BIM_Def8.npy


In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf1.predict(X)
    y_pred = second_encoder.inverse_transform(y_pred)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude3Attack/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude3Attack/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [    15      6    413    625      3     13      3 112889      2      2]
(15, 56) (15,)
Save 0 to baseline_Def1.npy
(6, 56) (6,)
Save 1 to baseline_Def2.npy
(413, 56) (413,)
Save 2 to baseline_Def3.npy
(625, 56) (625,)
Save 3 to baseline_Def4.npy
(3, 56) (3,)
Save 4 to baseline_Def5.npy
(13, 56) (13,)
Save 5 to baseline_Def6.npy
(3, 56) (3,)
Save 7 to baseline_Def8.npy
(112889, 56) (112889,)
Save 8 to baseline_Def9.npy
(2, 56) (2,)
Save 9 to baseline_Def10.npy
(2, 56) (2,)
Save 10 to baseline_Def11.npy
Execution Time: 0.773703 seconds
(341913,)
[ 0  1  2  3  4  5  8  9 10] [219859    383    209   1426     91   3795 116143      1      6]
(219859, 56) (219859,)
Save 0 to BIM_Def1.npy
(383, 56) (383,)
Save 1 to BIM_Def2.npy
(209, 56) (209,)
Save 2 to BIM_Def3.npy
(1426, 56) (1426,)
Save 3 to BIM_Def4.npy
(91, 56) (91,)
Save 4 to BIM_Def5.npy
(3795, 56) (3795,)
Save 5 to BIM_Def6.npy
(116143, 56) (116143,)
Save 8 to BIM_Def9.npy
(1, 56) (1,)
Save 9 

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt1.predict(X)
    y_pred = second_encoder.inverse_transform(y_pred)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude3Attack/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude3Attack/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [   689    892   1685   2711     18    944    716 105958    278     80]
(689, 56) (689,)
Save 0 to baseline_Def1.npy
(892, 56) (892,)
Save 1 to baseline_Def2.npy
(1685, 56) (1685,)
Save 2 to baseline_Def3.npy
(2711, 56) (2711,)
Save 3 to baseline_Def4.npy
(18, 56) (18,)
Save 4 to baseline_Def5.npy
(944, 56) (944,)
Save 5 to baseline_Def6.npy
(716, 56) (716,)
Save 7 to baseline_Def8.npy
(105958, 56) (105958,)
Save 8 to baseline_Def9.npy
(278, 56) (278,)
Save 9 to baseline_Def10.npy
(80, 56) (80,)
Save 10 to baseline_Def11.npy
Execution Time: 0.061497 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [215633   2419  29341  20367   1554  12058    488  59439    500    114]
(215633, 56) (215633,)
Save 0 to BIM_Def1.npy
(2419, 56) (2419,)
Save 1 to BIM_Def2.npy
(29341, 56) (29341,)
Save 2 to BIM_Def3.npy
(20367, 56) (20367,)
Save 3 to BIM_Def4.npy
(1554, 56) (1554,)
Save 4 to BIM_Def5.npy
(12058, 56) (12058,)
Save 5 to BIM_Def6.npy
(488, 56) (488,)
S

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_Exclude3Attack/UNSW_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_Exclude3Attack.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,87,114,1250,1510,6,138,0,26,110810,19,11,113971
1,XGB1,225056,170,365,2673,562,5374,0,85,107601,11,16,341913
2,XGB1,227154,89,226,2639,740,3984,0,202,106868,9,2,341913
3,XGB1,225056,170,365,2673,562,5374,0,85,107601,11,16,341913
4,XGB1,311690,2394,335,5579,165,1533,0,3,20168,5,41,341913
5,XGB1,206181,331,2248,4546,391,1207,0,44,126893,44,28,341913
6,XGB1,919,299,3234,4270,18,362,0,82,332647,49,33,341913
7,XGB1,72387,6,221,796,6,62,0,23,268403,0,9,341913
8,XGB1,219778,339,397,3532,239,3721,0,80,113798,1,28,341913
9,XGB1,221818,119,761,3226,375,5924,0,183,109479,10,18,341913
